### SQliteのbrat形式のデータからアノテーションを取得してbottleで可視化する

#### SQlite接続の関数定義

In [1]:
import json
import sqlite3

conn = None

# sqlite3に接続
def connect():
    # global変数でconnを呼び出し
    global conn
    # データベースの場所を指定
    conn = sqlite3.connect('./data/sqlite3/sqlite3')

# 一部のデータを見る
def sqlite3_get(doc_id, fl):
    # .fetchone()で一致した内の一行を取得する
    row_ls = conn.execute(f"SELECT {','.join(fl)} FROM docs WHERE id = {doc_id}").fetchone()
    row_dict = {}
    # flとrow_lsで抜き出したデータをzipする
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

# sqlite3を切断
def close():
    conn.close()

# アノテーションを取得
def get_annotation(doc_id, name):
    # docsのid行をwhere idで指定しnameから取り出す
    row = conn.execute('SELECT {0} FROM docs WHERE id = ?'.format(name),(doc_id,)).fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []

###  bottleの記述

In [ ]:
import json
import re

import bottle

# ルートディレクトリ設定
@bottle.route('/')
def index_html():
    return bottle.static_file('SQlitepracticeInbottle.html', root='./data/static/')

# その他パス設定
@bottle.route('/file/<filename:path>')
def static(filename):
    return bottle.static_file(filename, root='./data/static/')

@bottle.get('/get')
def get():
    # パラメータのidを取得
    doc_id = bottle.request.params.id
    
    # パラメータのnames配列を取得
    # namesはアノテーション名が空白区切りで連結されている事を想定している
    # splitで分割しアノテーション名のリストに変換している
    names = bottle.request.params.names.split()

    print('doc_id:' + doc_id)
    
    # sqlite３に接続し、id行からcontent列のデータを抜き出す
    row = sqlite3_get(doc_id, fl=['content'])
    text = row['content']
    # text = re.sub(r'[。 !', '\n', text)
    
    # 下記でbratのデータ型を作って置く
    data = {
        'collection': {
            'entity_types':     [],
        },
        'annotation': {
            'text':             text,
            'entities':         [],
            'relations':        [],
        },
    }

    mapping = {}
    for name in names:
        # sqlite3からアノテーションを取得
        annos = get_annotation(doc_id, name)
        # アノテーションを番号順をつけ抜き出し
        # dataにappendで抜き出したデータを入れ込む
        for i, anno in enumerate(annos):
            data['collection']['entity_types'].append({
                'type':        name,
                'bgColor':     '#7fa2ff',
                'borderColor': 'darken'
            })

            Ti = 'T{0:d}'.format(len(data['annotation']['entities']) + 1)
            data['annotation']['entities'].append([
                Ti,
                name,
                [[anno['begin'], anno['end']]]
            ])
            mapping[(name, i)] = Ti
    # linkの情報を扱うためにmapping変数を定義し、アノテーション名とアノテーションIDからTiを呼び出せるようにしている
    for name in names:
        annos = get_annotation(doc_id, name)
        for i, anno in enumerate(annos):
            if 'link' not in anno:
                continue
            name_linked, i_linked = anno['link']
            if (name, i) not in mapping or (name_linked, i_linked) not in mapping:
                continue

            data['annotation']['relations'].append([
                'R{0:d}'.format(len(data['annotation']['relations']) + 1),
                'arg',
                [['src', mapping[(name, i)]], ['tgt', mapping[(name_linked, i_linked)]]]
            ])

    return json.dumps(data, ensure_ascii=False)


connect()
bottle.run(host='localhost', port='8702')
close()